# Scaled Dot-Product Attention\n\n## Introduction\n\nBienvenue dans ce notebook sur le **mécanisme d'attention**, le cœur des transformers!\n\n### Objectifs pédagogiques\n\nDans ce notebook, vous allez:\n1. Comprendre la formule mathématique de l'attention\n2. Implémenter l'attention from scratch avec NumPy\n3. Implémenter l'attention avec PyTorch\n4. Visualiser les poids d'attention\n5. Comprendre le rôle de chaque composant (Q, K, V)\n\n### Qu'est-ce que l'attention?\n\nL'attention est un mécanisme qui permet à chaque token de \"regarder\" les autres tokens et de pondérer leur importance. C'est comme si chaque mot dans une phrase pouvait décider quels autres mots sont importants pour comprendre son sens.\n\n**Exemple concret:**\nDans la phrase \"Le chat qui était sur le tapis a mangé la souris\", le mot \"mangé\" doit prêter attention à:\n- \"chat\" (qui a mangé?)\n- \"souris\" (qu'est-ce qui a été mangé?)\n\nL'attention calcule automatiquement ces relations!

## 1. Formule Mathématique de l'Attention\n\n### Formule Complète\n\n$$\\text{Attention}(Q, K, V) = \\text{softmax}\\left(\\frac{QK^T}{\\sqrt{d_k}}\\right)V$$\n\n### Décomposition Étape par Étape\n\n#### Étape 1: Calcul des Scores de Similarité\n\n$$S = QK^T$$\n\nOù:\n- $Q \\in \\mathbb{R}^{n \\times d_k}$ : Matrice des **Queries** (\"ce que je cherche\")\n- $K \\in \\mathbb{R}^{m \\times d_k}$ : Matrice des **Keys** (\"ce que j'offre comme information\")\n- $S \\in \\mathbb{R}^{n \\times m}$ : Matrice des **scores** de similarité\n\n**Intuition:** $S_{ij}$ mesure la similarité entre la query $i$ et la key $j$.\n\n#### Étape 2: Normalisation (Scaling)\n\n$$S' = \\frac{S}{\\sqrt{d_k}}$$\n\n**Pourquoi diviser par $\\sqrt{d_k}$?**\n- Les produits scalaires croissent avec la dimension $d_k$\n- Sans normalisation, le softmax sature (gradients $\\rightarrow 0$)\n- $\\sqrt{d_k}$ est la déviation standard théorique du produit scalaire\n\n#### Étape 3: Calcul des Poids d'Attention\n\n$$A = \\text{softmax}(S')$$\n\nOù le softmax est défini comme:\n\n$$\\text{softmax}(x_i) = \\frac{e^{x_i}}{\\sum_{j=1}^m e^{x_j}}$$\n\n**Propriétés du softmax:**\n- Toutes les valeurs sont entre 0 et 1\n- La somme de chaque ligne = 1 (distribution de probabilité)\n- Les scores élevés $\\rightarrow$ probabilités élevées\n\n#### Étape 4: Pondération des Valeurs\n\n$$O = AV$$\n\nOù:\n- $V \\in \\mathbb{R}^{m \\times d_v}$ : Matrice des **Values** (\"l'information elle-même\")\n- $O \\in \\mathbb{R}^{n \\times d_v}$ : Sortie pondérée\n\n**Intuition:** Chaque ligne de $O$ est une somme pondérée des valeurs, où les poids sont donnés par $A$.\n\n$$O_i = \\sum_{j=1}^m A_{ij} V_j$$

In [ ]:
# Imports\nimport numpy as np\nimport math\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nimport sys\nfrom typing import Tuple, Optional\n\n# Ajouter le chemin vers src/\nsys.path.append('../..')\n\n# Configuration pour les visualisations\nplt.style.use('default')\nsns.set_palette(\"husl\")\n\n# Seed pour la reproductibilité\nnp.random.seed(42)\ntorch.manual_seed(42)\n\nprint(\"✓ Imports réussis!\")\nprint(f\"✓ PyTorch version: {torch.__version__}\")\nprint(f\"✓ Device disponible: {'GPU' if torch.cuda.is_available() else 'CPU'}\")

## 2. Implémentation From Scratch (NumPy)\n\nCommençons par implémenter l'attention avec NumPy pour comprendre chaque opération mathématique.\n\n### 2.1 Fonction Softmax

In [ ]:
def softmax_from_scratch(x: np.ndarray, axis: int = -1) -> np.ndarray:\n    \"\"\"\n    Implémentation from-scratch du softmax pour la stabilité numérique.\n    \n    Formule: softmax(x_i) = exp(x_i) / sum(exp(x_j))\n    \n    Astuce de stabilité: softmax(x) = softmax(x - max(x))\n    \"\"\"\n    # Soustraire le max pour la stabilité numérique (évite overflow)\n    x_shifted = x - np.max(x, axis=axis, keepdims=True)\n    \n    # Calculer les exponentielles\n    exp_x = np.exp(x_shifted)\n    \n    # Normaliser pour obtenir des probabilités\n    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)\n\n# Test du softmax\ntest_scores = np.array([1.0, 2.0, 3.0])\ntest_probs = softmax_from_scratch(test_scores)\nprint(\"Scores:\", test_scores)\nprint(\"Probabilités:\", test_probs)\nprint(\"Somme:\", test_probs.sum())\nprint(\"✓ Le softmax fonctionne!\")

### 2.2 Attention Complète From Scratch

In [ ]:
def scaled_dot_product_attention_from_scratch(\n    Q: np.ndarray,\n    K: np.ndarray,\n    V: np.ndarray,\n    mask: Optional[np.ndarray] = None\n) -> Tuple[np.ndarray, np.ndarray]:\n    \"\"\"\n    Implémentation from-scratch de l'attention avec NumPy.\n    \n    Args:\n        Q: Query matrix (batch_size, seq_len, d_k)\n        K: Key matrix (batch_size, seq_len, d_k)\n        V: Value matrix (batch_size, seq_len, d_v)\n        mask: Optional mask (seq_len, seq_len)\n    \n    Returns:\n        output: (batch_size, seq_len, d_v)\n        attention_weights: (batch_size, seq_len, seq_len)\n    \"\"\"\n    d_k = Q.shape[-1]\n    \n    # ÉTAPE 1: Calcul des scores (QK^T)\n    scores = np.matmul(Q, K.transpose(0, 2, 1))\n    print(f\"[1] Scores shape après QK^T: {scores.shape}\")\n    \n    # ÉTAPE 2: Normalisation (scaling)\n    scores = scores / math.sqrt(d_k)\n    print(f\"[2] Scores après scaling par sqrt({d_k}) = {math.sqrt(d_k):.2f}\")\n    \n    # ÉTAPE 3: Application du masque (optionnel)\n    if mask is not None:\n        scores = np.where(mask == 0, -1e9, scores)\n        print(f\"[3] Masque appliqué\")\n    \n    # ÉTAPE 4: Calcul des poids d'attention (softmax)\n    attention_weights = softmax_from_scratch(scores, axis=-1)\n    print(f\"[4] Poids d'attention shape: {attention_weights.shape}\")\n    print(f\"[4] Vérification normalisation: sum = {attention_weights[0, 0, :].sum():.4f}\")\n    \n    # ÉTAPE 5: Pondération des valeurs (AV)\n    output = np.matmul(attention_weights, V)\n    print(f\"[5] Output shape: {output.shape}\")\n    \n    return output, attention_weights\n\nprint(\"✓ Fonction d'attention from-scratch définie!\")

### 2.3 Exemple avec Petites Matrices (3×3)\n\nTestons notre implémentation avec un exemple simple: 3 tokens avec dimension d_k = 4.

In [ ]:
# Paramètres pour l'exemple\nbatch_size = 1\nseq_len = 3  # 3 tokens\nd_k = 4      # dimension des clés/queries\nd_v = 4      # dimension des valeurs\n\n# Créer des matrices Q, K, V simples\nnp.random.seed(42)\nQ = np.random.randn(batch_size, seq_len, d_k)\nK = np.random.randn(batch_size, seq_len, d_k)\nV = np.random.randn(batch_size, seq_len, d_v)\n\nprint(\"=\" * 60)\nprint(\"EXEMPLE: Attention avec 3 tokens\")\nprint(\"=\" * 60)\nprint(f\"\\nInput shapes:\")\nprint(f\"  Q (Query):  {Q.shape} - 'Ce que je cherche'\")\nprint(f\"  K (Key):    {K.shape} - 'Ce que j'offre'\")\nprint(f\"  V (Value):  {V.shape} - 'L'information'\")\nprint()

In [ ]:
# Calculer l'attention\noutput, attention_weights = scaled_dot_product_attention_from_scratch(Q, K, V)\n\nprint(\"\\n\" + \"=\" * 60)\nprint(\"RÉSULTATS\")\nprint(\"=\" * 60)\nprint(f\"\\nOutput shape: {output.shape}\")\nprint(f\"Attention weights shape: {attention_weights.shape}\")\n\nprint(\"\\nAttention weights (batch 0):\")\nprint(attention_weights[0])\n\nprint(\"\\nInterprétation:\")\nprint(\"  - Ligne i = distribution d'attention du token i\")\nprint(\"  - Colonne j = combien les autres tokens regardent le token j\")\nprint(\"  - Chaque ligne somme à 1.0 (probabilités)\")\n\n# Vérifier la normalisation\nrow_sums = attention_weights.sum(axis=-1)\nprint(f\"\\nVérification: somme de chaque ligne = {row_sums[0]}\")\nprint(f\"Toutes les sommes ≈ 1.0? {np.allclose(row_sums, 1.0)}\")

## 3. Implémentation PyTorch (Professionnelle)\n\nMaintenant, implémentons l'attention avec PyTorch en utilisant les opérations optimisées.\n\n### 3.1 Classe ScaledDotProductAttention

In [ ]:
class ScaledDotProductAttention(nn.Module):\n    \"\"\"\n    Implémentation PyTorch professionnelle de l'attention.\n    \n    Méthodes PyTorch utilisées:\n    - torch.matmul(): Multiplication matricielle optimisée GPU\n    - masked_fill(): Remplace les valeurs selon un masque\n    - F.softmax(): Softmax stable numériquement\n    \"\"\"\n    \n    def __init__(self, d_k: int):\n        super().__init__()\n        self.d_k = d_k\n        self.scale = 1.0 / math.sqrt(d_k)\n    \n    def forward(\n        self,\n        Q: torch.Tensor,\n        K: torch.Tensor,\n        V: torch.Tensor,\n        mask: Optional[torch.Tensor] = None\n    ) -> Tuple[torch.Tensor, torch.Tensor]:\n        \"\"\"\n        Calcule l'attention Scaled Dot-Product.\n        \n        Args:\n            Q: Query tensor (batch_size, seq_len, d_k)\n            K: Key tensor (batch_size, seq_len, d_k)\n            V: Value tensor (batch_size, seq_len, d_v)\n            mask: Optional mask (seq_len, seq_len)\n        \n        Returns:\n            output: (batch_size, seq_len, d_v)\n            attention_weights: (batch_size, seq_len, seq_len)\n        \"\"\"\n        \n        # ÉTAPE 1: Calcul des scores (QK^T)\n        # torch.matmul() gère automatiquement le batching\n        scores = torch.matmul(Q, K.transpose(-2, -1))\n        \n        # ÉTAPE 2: Normalisation (scaling)\n        scores = scores * self.scale\n        \n        # ÉTAPE 3: Application du masque (optionnel)\n        if mask is not None:\n            # masked_fill() remplace les positions masquées par -inf\n            scores = scores.masked_fill(mask == 0, float('-inf'))\n        \n        # ÉTAPE 4: Calcul des poids d'attention (softmax)\n        # F.softmax() avec dim=-1 normalise sur la dernière dimension\n        attention_weights = F.softmax(scores, dim=-1)\n        \n        # ÉTAPE 5: Pondération des valeurs (AV)\n        output = torch.matmul(attention_weights, V)\n        \n        return output, attention_weights\n\nprint(\"✓ Classe ScaledDotProductAttention définie!\")

### 3.2 Exemple avec PyTorch

In [ ]:
# Créer des tensors PyTorch\ntorch.manual_seed(42)\nQ_torch = torch.randn(batch_size, seq_len, d_k)\nK_torch = torch.randn(batch_size, seq_len, d_k)\nV_torch = torch.randn(batch_size, seq_len, d_v)\n\nprint(\"=\" * 60)\nprint(\"EXEMPLE: Attention avec PyTorch\")\nprint(\"=\" * 60)\nprint(f\"\\nInput shapes (PyTorch tensors):\")\nprint(f\"  Q: {Q_torch.shape}\")\nprint(f\"  K: {K_torch.shape}\")\nprint(f\"  V: {V_torch.shape}\")\nprint(f\"  Device: {Q_torch.device}\")\n\n# Créer le module d'attention\nattention = ScaledDotProductAttention(d_k=d_k)\n\n# Calculer l'attention\noutput_torch, attention_weights_torch = attention(Q_torch, K_torch, V_torch)\n\nprint(f\"\\nOutput shapes:\")\nprint(f\"  Output: {output_torch.shape}\")\nprint(f\"  Attention weights: {attention_weights_torch.shape}\")\n\nprint(\"\\nAttention weights (batch 0):\")\nprint(attention_weights_torch[0])\n\n# Vérifier la normalisation\nrow_sums = attention_weights_torch.sum(dim=-1)\nprint(f\"\\nVérification: somme de chaque ligne (batch 0):\")\nprint(row_sums[0])\nprint(f\"Toutes les sommes ≈ 1.0? {torch.allclose(row_sums, torch.ones_like(row_sums))}\")

## 4. Visualisation des Poids d'Attention\n\nVisualisons les poids d'attention sous forme de heatmap pour mieux comprendre quels tokens prêtent attention à quels autres tokens.

In [ ]:
def visualize_attention(attention_weights, tokens=None, title=\"Poids d'Attention\"):\n    \"\"\"\n    Visualise les poids d'attention sous forme de heatmap.\n    \n    Args:\n        attention_weights: Matrice d'attention (seq_len, seq_len)\n        tokens: Liste optionnelle de tokens pour les labels\n        title: Titre du graphique\n    \"\"\"\n    plt.figure(figsize=(8, 6))\n    \n    # Créer la heatmap\n    if tokens is None:\n        tokens = [f\"Token {i}\" for i in range(len(attention_weights))]\n    \n    sns.heatmap(\n        attention_weights,\n        xticklabels=tokens,\n        yticklabels=tokens,\n        cmap='YlOrRd',\n        annot=True,\n        fmt='.3f',\n        cbar_kws={'label': 'Poids d\\'attention'},\n        vmin=0,\n        vmax=1\n    )\n    \n    plt.xlabel('Keys (ce qu\\'on regarde)', fontsize=12)\n    plt.ylabel('Queries (qui regarde)', fontsize=12)\n    plt.title(title, fontsize=14, fontweight='bold')\n    plt.tight_layout()\n    plt.show()\n\n# Visualiser les poids d'attention de notre exemple\ntokens_example = [\"Token 0\", \"Token 1\", \"Token 2\"]\nvisualize_attention(\n    attention_weights_torch[0].detach().numpy(),\n    tokens=tokens_example,\n    title=\"Poids d'Attention - Exemple 3 Tokens\"\n)

## 5. Attention avec Masque Causal (GPT)\n\nLe masque causal empêche les tokens de voir le futur. C'est essentiel pour les modèles génératifs comme GPT.\n\n### 5.1 Création du Masque Causal

In [ ]:
# Créer un masque causal avec torch.tril()\nseq_len_mask = 5\ncausal_mask = torch.tril(torch.ones(seq_len_mask, seq_len_mask))\n\nprint(\"Masque causal (1=autorisé, 0=bloqué):\")\nprint(causal_mask)\nprint(\"\\nInterprétation:\")\nprint(\"  - Chaque ligne représente un token\")\nprint(\"  - 1 = peut voir ce token, 0 = ne peut pas voir\")\nprint(\"  - Token i peut voir tokens 0 à i (autorégressif)\")\n\n# Visualiser le masque\nplt.figure(figsize=(6, 5))\nsns.heatmap(\n    causal_mask.numpy(),\n    cmap='RdYlGn',\n    annot=True,\n    fmt='.0f',\n    cbar=False,\n    square=True\n)\nplt.title('Masque Causal (Triangulaire Inférieur)', fontsize=14, fontweight='bold')\nplt.xlabel('Position Key')\nplt.ylabel('Position Query')\nplt.tight_layout()\nplt.show()

### 5.2 Comparaison: Avec et Sans Masque Causal

In [ ]:
# Créer des données de test\ntorch.manual_seed(42)\nQ_test = torch.randn(1, seq_len_mask, 8)\nK_test = torch.randn(1, seq_len_mask, 8)\nV_test = torch.randn(1, seq_len_mask, 8)\n\nattention_test = ScaledDotProductAttention(d_k=8)\n\n# Attention SANS masque (bidirectionnelle - BERT)\nprint(\"=\" * 60)\nprint(\"Attention SANS masque (bidirectionnelle - BERT)\")\nprint(\"=\" * 60)\noutput_no_mask, weights_no_mask = attention_test(Q_test, K_test, V_test, mask=None)\nprint(\"\\nPoids d'attention (tous les tokens se voient):\")\nprint(weights_no_mask[0])\n\n# Attention AVEC masque causal (autorégressif - GPT)\nprint(\"\\n\" + \"=\" * 60)\nprint(\"Attention AVEC masque causal (autorégressif - GPT)\")\nprint(\"=\" * 60)\noutput_masked, weights_masked = attention_test(Q_test, K_test, V_test, mask=causal_mask)\nprint(\"\\nPoids d'attention (masque causal appliqué):\")\nprint(weights_masked[0])\nprint(\"\\nObservation:\")\nprint(\"  - La partie supérieure droite est nulle (pas d'attention au futur)\")\nprint(\"  - Chaque token ne voit que lui-même et les tokens précédents\")\nprint(\"  - C'est le mécanisme clé de GPT pour la génération autoregressive\")

In [ ]:
# Visualiser la comparaison\nfig, axes = plt.subplots(1, 2, figsize=(14, 5))\n\n# Sans masque\nsns.heatmap(\n    weights_no_mask[0].detach().numpy(),\n    ax=axes[0],\n    cmap='YlOrRd',\n    annot=True,\n    fmt='.3f',\n    cbar_kws={'label': 'Poids'},\n    vmin=0,\n    vmax=1\n)\naxes[0].set_title('BERT: Attention Bidirectionnelle\\n(Tous les tokens se voient)', fontweight='bold')\naxes[0].set_xlabel('Keys')\naxes[0].set_ylabel('Queries')\n\n# Avec masque\nsns.heatmap(\n    weights_masked[0].detach().numpy(),\n    ax=axes[1],\n    cmap='YlOrRd',\n    annot=True,\n    fmt='.3f',\n    cbar_kws={'label': 'Poids'},\n    vmin=0,\n    vmax=1\n)\naxes[1].set_title('GPT: Attention Causale\\n(Pas de vision du futur)', fontweight='bold')\naxes[1].set_xlabel('Keys')\naxes[1].set_ylabel('Queries')\n\nplt.tight_layout()\nplt.show()\n\n# Statistiques\nprint(\"\\nComparaison: Nombre de tokens visibles par position\")\nprint(\"=\" * 60)\nfor i in range(seq_len_mask):\n    visible_no_mask = (weights_no_mask[0, i] > 0).sum().item()\n    visible_masked = (weights_masked[0, i] > 0).sum().item()\n    print(f\"  Position {i}: Sans masque={visible_no_mask}, Avec masque={visible_masked}\")

## 6. Exercices Pratiques\n\n### Exercice 1: Calcul Manuel\n\nCalculez manuellement l'attention pour cet exemple simple:\n\n```\nQ = [[1, 0]]\nK = [[1, 0], [0, 1]]\nV = [[2, 0], [0, 3]]\nd_k = 2\n```\n\n**Étapes:**\n1. Calculez $QK^T$\n2. Divisez par $\\sqrt{d_k} = \\sqrt{2}$\n3. Appliquez softmax\n4. Multipliez par V\n\n**TODO:** Complétez le code ci-dessous pour vérifier votre calcul.

In [ ]:
# TODO: Exercice 1 - Calcul manuel\n# Complétez ce code pour vérifier votre calcul manuel\n\nQ_ex1 = np.array([[[1, 0]]])  # (1, 1, 2)\nK_ex1 = np.array([[[1, 0], [0, 1]]])  # (1, 2, 2)\nV_ex1 = np.array([[[2, 0], [0, 3]]])  # (1, 2, 2)\n\n# Votre calcul manuel ici:\n# Étape 1: QK^T = ?\n# Étape 2: Scaling = ?\n# Étape 3: Softmax = ?\n# Étape 4: Output = ?\n\n# Vérification avec notre fonction\noutput_ex1, weights_ex1 = scaled_dot_product_attention_from_scratch(Q_ex1, K_ex1, V_ex1)\nprint(\"Poids d'attention:\", weights_ex1[0])\nprint(\"Output:\", output_ex1[0])

### Exercice 2: Expérimentation avec d_k\n\nObservez l'effet de la dimension $d_k$ sur les poids d'attention.\n\n**Question:** Que se passe-t-il quand $d_k$ augmente? Pourquoi la normalisation par $\\sqrt{d_k}$ est-elle importante?

In [ ]:
# TODO: Exercice 2 - Effet de d_k\n# Testez avec différentes valeurs de d_k: 4, 16, 64, 256\n\ndef test_different_dk(d_k_values):\n    \"\"\"Teste l'attention avec différentes valeurs de d_k.\"\"\"\n    for d_k in d_k_values:\n        torch.manual_seed(42)\n        Q = torch.randn(1, 3, d_k)\n        K = torch.randn(1, 3, d_k)\n        V = torch.randn(1, 3, d_k)\n        \n        attention = ScaledDotProductAttention(d_k=d_k)\n        _, weights = attention(Q, K, V)\n        \n        # Calculer la variance des poids\n        variance = weights.var().item()\n        print(f\"d_k={d_k:3d}: variance des poids = {variance:.6f}\")\n\nprint(\"Effet de d_k sur la distribution des poids d'attention:\")\ntest_different_dk([4, 16, 64, 256])\n\nprint(\"\\nObservation: Sans scaling, les poids deviendraient trop concentrés!\")

## 7. Résumé\n\n### Ce que nous avons appris\n\n1. **Formule de l'attention:** $\\text{Attention}(Q, K, V) = \\text{softmax}\\left(\\frac{QK^T}{\\sqrt{d_k}}\\right)V$\n\n2. **Les 4 étapes:**\n   - Calcul des scores de similarité (QK^T)\n   - Normalisation par $\\sqrt{d_k}$\n   - Application du softmax pour obtenir des probabilités\n   - Pondération des valeurs\n\n3. **Rôle de Q, K, V:**\n   - **Q (Query):** \"Ce que je cherche\"\n   - **K (Key):** \"Ce que j'offre comme information\"\n   - **V (Value):** \"L'information elle-même\"\n\n4. **Masque causal:**\n   - Empêche de voir le futur\n   - Essentiel pour GPT (génération autoregressive)\n   - BERT n'utilise pas de masque (attention bidirectionnelle)\n\n### Prochaines étapes\n\nDans le prochain notebook, nous verrons:\n- **Multi-Head Attention:** Plusieurs têtes d'attention en parallèle\n- Comment combiner plusieurs perspectives\n- L'architecture complète du transformer\n\n### Points clés à retenir\n\n✓ L'attention permet à chaque token de \"regarder\" les autres tokens\n✓ Le scaling par $\\sqrt{d_k}$ est crucial pour la stabilité\n✓ Le softmax convertit les scores en probabilités\n✓ Le masque causal est la différence clé entre BERT et GPT